In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install ydata-profiling

In [ ]:
import os
import pandas as pd

os.listdir("/kaggle/input/satire-detection/")
# df = pd.read_csv("/kaggle/input/satire-detection/train.csv")


In [ ]:
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/satire-detection/random_seed_setter.py", dst = "../working/seed_setter.py")

# import all our functions
from seed_setter import *

In [ ]:
set_random_seeds()

In [265]:
import pandas
import numpy
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import spacy
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
 

In [251]:
train_df = pd.read_csv("/kaggle/input/satire-detection/train.csv")
test_df = pd.read_csv("/kaggle/input/satire-detection/test.csv")

X = train_df[['title', 'content']]
y = train_df['class']

In [ ]:
from ydata_profiling import ProfileReport

train_profile = ProfileReport(train_df, title='Train Profiling Report', explorative=True)
train_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y , train_size=0.8, stratify = y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train , train_size=0.9, stratify = y_train)


In [ ]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_valid))
print(len(y_valid))


In [ ]:
HTML_CLEANER = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
PUNCT_CLEANER = re.compile(r"[^\w\s]|_")
def preprocess(text):
    
    if isinstance(text, str):
        # remove HTML tags 
        text = re.sub(CLEANER, '', text)
        
        # remove punctuation
        text = re.sub(PUNCT_CLEANER, '', text)

        # Normalize the data
        text = text.lower()

        # remove spaces at the beginning or at the end
        text = text.strip()

        #remove multiple spaces in a row

        text = re.sub(' +', ' ', text)
        #lematize the data

        # specifically for the romanian tokenizer
        text = text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")
    
       
        return text
    else:
        return " "



In [ ]:
X_test.head()

In [257]:
X_train['title'] = X_train['title'].apply(preprocess) 
X_train['content'] = X_train['content'].apply(preprocess) 
X_test['title'] =  X_test['title'].apply(preprocess) 
X_test['content'] = X_test['content'].apply(preprocess) 
X_valid['title'] = X_valid['title'].apply(preprocess) 
X_valid['content'] = X_valid['content'].apply(preprocess) 

In [258]:
X_valid.head()

,title,content
40220,dumitru buzatu unul dintre liderii psd care pr...,președintele psd vaslui dumitru buzatu a decla...
38325,peste 12000 de alergatori la omv petrom buchar...,peste 12000 de oameni din peste 50 de țări au ...
18948,un perete cu faianță semnat dedeman vândut cu ...,arta românească vinde în continuare după ce br...
60944,alianța usrcovid face jocurile la locale,iohannis și pnl au susținut începerea școlii î...
70069,generalul opriș șeful sts ia de la stat și dă ...,degeaba are ministerul administrației și inter...


In [216]:
!python -m spacy download ro_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 30.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_sm')


In [263]:
nlp = spacy.load("ro_core_news_sm", disable=["parser", "ner"])  # Disable unnecessary components to speed up

In [274]:
def get_embeddings(df):
    embeddings = {}
    for column in df:
        emb_list = []
        for doc in nlp.pipe(df[column].astype(str), batch_size=50):  # Ensure text input with .astype(str)
            valid_embs = [token.vector for token in doc if np.any(token.vector)]
            
            if valid_embs:
                mean_emb = np.mean(valid_embs, axis=0)
            else:
                mean_emb = np.zeros((nlp.vocab.vectors_length,))
                
            emb_list.append(mean_emb)

        embeddings[f"{column}_embeddings"] = np.array(emb_list)

    return embeddings


In [276]:
# train_emb = get_embeddings(X_train)
test_emb = get_embeddings(X_test)
valid_emb = get_embeddings(X_valid)

In [297]:
def train_random_forest(emb, labels):
    classifier = RandomForestClassifier(n_estimators = 100, random_state = 42, n_jobs = -1)
    classifier.fit(emb, labels)
    
    return classifier

    

In [298]:
y_valid = np.array(y_valid).astype(int)
y_valid
model = train_random_forest(valid_emb['title_embeddings'], y_valid)

In [299]:
test_emb = get_embeddings(X_test)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (14115,) + inhomogeneous part.

In [ ]:
test_res = model.predict(test_emb)

In [ ]:
acc = balanced_accuracy_score(np.array(y_test).astype(int), test_res)